In [ ]:
import pandas as pd
import geopandas as gpd

1. Obtener ubicaciones - cuando los pagos fueron realizados por Diego
2. Obtener correos electrónicos de transacciones, a partir del 2025-01-01
3. Obtener transacciones de Splitwise, incluir las transacciones eliminadas, pero mantener solo las más recientes con el mismo precio y descripción/hash.
   Conservar solo las creadas por Diego.



In [ ]:
def clean_datetime_serie(serie: pd.Series, timezone="America/Santiago") -> pd.Series:
    """
    Cleans a datetime column by converting it to a standard format.
    """
    new = pd.to_datetime(
        serie, format="ISO8601", utc=True, errors="coerce"
    ).dt.tz_convert(timezone)
    return new

In [ ]:
def read_locations(file_name: str) -> gpd.GeoDataFrame:
    assert file_name.endswith(".jsonl"), "File must be a JSON Lines file"
    df = pd.read_json(file_name, lines=True)
    df["date"] = clean_datetime_serie(df["date"])
    df = df.reset_index(drop=False, names=["id"])

    gdf = gpd.GeoDataFrame(
        df,
        geometry=gpd.points_from_xy(df["lon"], df["lat"]),
        crs="EPSG:4326",
    )

    drop_columns = ["lon", "lat", "transaction"]
    drop_columns = [col for col in drop_columns if col in df.columns]
    gdf = gdf.drop(columns=drop_columns)

    gdf = gdf.rename(columns=lambda x: f"location_{x}" if x not in ["geometry"] else x)
    return gdf


locations = read_locations("raw/locations.jsonl")
locations.plot()

In [ ]:
locations

In [ ]:
from typing import Optional

EXPENSES_CLEANED_COLUMNS = [
    "id",
    "date",
    "category_name",
    "description",
    "cost",
    "details",
    "created_at",
    "updated_at",
    "deleted_at",
    "created_by_name",
    "updated_by_name",
    "deleted_by_name",
]


def is_duplicated_expense(expenses: pd.DataFrame) -> pd.Series:
    # Un expense se considera duplicado si tiene la misma fecha, costo y descripción
    # Marcaremos como duplicados a los que tienen una fecha de deleted_at no nula (que fueron eliminados)
    # cols = ["date", "cost", "description"]
    cols = [
        "date",
        "cost",
    ]  # I changed the description of some expenses when testing, but they are the same in the end...
    duplicated = (
        expenses.groupby(cols)
        .agg(cnt=("id", "count"), ids=("id", list))
        .pipe(lambda df: df[df.cnt > 1])
    )
    duplicated_ids = duplicated["ids"].explode().unique()
    is_duplicated = expenses["id"].isin(duplicated_ids) & ~expenses["deleted_at"].isna()
    return is_duplicated


def read_expenses(
    file_name: str,
    keep_all_columns: Optional[bool] = False,
    keep_duplicated: Optional[bool] = True,
) -> pd.DataFrame:
    assert file_name.endswith(".json"), "File must be a JSON file"

    df = pd.read_json(file_name)
    for col in ["date", "created_at", "updated_at", "deleted_at"]:
        df[col] = clean_datetime_serie(df[col])

    for col in ["created_by", "updated_by", "deleted_by"]:
        df[f"{col}_id"] = df[col].str["id"]
        df[f"{col}_name"] = df[col].str["first_name"]

    df["category_id"] = df["category"].str["id"]
    df["category_name"] = df["category"].str["name"]

    if not keep_all_columns:
        df = df[EXPENSES_CLEANED_COLUMNS]

    is_duplicated = is_duplicated_expense(df)
    if keep_duplicated:
        df["is_duplicated"] = is_duplicated
    else:
        df = df[~is_duplicated]

    df = df.rename(columns=lambda x: f"expense_{x}")
    return df


expenses = read_expenses("raw/expenses.json", keep_all_columns=False)
expenses

In [ ]:
expenses[expenses.expense_is_duplicated].shape[0]

In [ ]:
expenses.groupby(["expense_date", "expense_cost"]).expense_id.count().pipe(
    lambda s: s[s > 1]
)

In [ ]:
expenses = expenses[~expenses.expense_is_duplicated]

In [ ]:
from functools import reduce
from typing import Callable


def apply_transforms(
    df: pd.DataFrame, transforms: Callable[[pd.DataFrame], pd.DataFrame]
) -> pd.DataFrame:
    df = reduce(
        lambda df, transform: transform(df),
        transforms,
        df,
    )
    return df

In [ ]:
def expand_dict_column(column_name: str) -> Callable[[pd.DataFrame], pd.DataFrame]:
    def inner(df: pd.DataFrame) -> pd.DataFrame:
        fill_na_values = lambda x: x if isinstance(x, dict) else {}
        df[column_name] = df[column_name].apply(fill_na_values)
        frame = (
            df[column_name]
            .apply(pd.Series)
            .rename(columns=lambda x: f"{column_name}_{x}")
        )
        return frame.join(df.drop(column_name, axis=1))

    return inner


TRANSACTION_CLEANED_COLUMNS = [
    "transaction_cost",
    "transaction_currency_code",
    "transaction_date",
    "transaction_description",
    "transaction_card_number",
    "transaction_hash",
]


def read_emails(
    filename: str, keep_all_columns: Optional[bool] = False, keep_errored: bool = False
) -> pd.DataFrame:
    assert filename.endswith(".json"), "File must be a JSON file"
    df = pd.read_json(filename)
    transforms = [expand_dict_column("email"), expand_dict_column("transaction")]
    frame = apply_transforms(df, transforms)
    for col in ["transaction_date", "email_date"]:
        frame[col] = clean_datetime_serie(frame[col])

    if not keep_errored:
        frame = frame[frame["error"].isna()]

    if not keep_all_columns:
        frame = frame[TRANSACTION_CLEANED_COLUMNS]
    return frame


transactions = read_emails("raw/emails.json")
transactions

In [ ]:
is_cajero = transactions["transaction_description"].str.contains(
    "con cargo a Cuenta ****4503", regex=False
)
transactions[is_cajero]

In [ ]:
transactions = transactions[~is_cajero]

In [ ]:
transactions["transaction_description"].unique()

In [ ]:
transactions.to_pickle("processed/transactions.pkl")
locations.to_file("processed/locations.geojson", driver="GeoJSON")
expenses.to_pickle("processed/expenses.pkl")

# Joined datasets with location and expenses

Cómo asociar transacciones con gastos
1. Muy fácil: usar los campos hash. Asumir que las transacciones eliminadas significan "gasto personal"
2. Asociar transacciones por día y con montos similares. Podemos asumir que cualquier transacción que no esté en Splitwise puede ser personal.

Estudiar:
1. ¿Existe alguna relación entre la descripción y el comerciante de la transacción? Es decir, ¿podemos crear una mejor descripción a partir del comerciante?
2. ¿Cuál es la relación entre las ubicaciones y los gastos? Cuánto aporta en predecir que es un gasto personal/compartido?

In [ ]:
def match_transactions(
    expenses_df: pd.DataFrame,
    transactions_df: pd.DataFrame,
    date_threshold: pd.Timedelta = pd.Timedelta("12h"),
    amount_threshold: float = 0.001,
) -> pd.DataFrame:
    # This is very slow, but it works
    # Need to rework if you want to use all your emails and expenses

    assert expenses_df["expense_id"].is_unique, "Expenses ids are not unique"
    assert transactions_df[
        "transaction_hash"
    ].is_unique, "Transactions hashes are not unique"

    expenses = expenses_df.copy()
    transactions = transactions_df.copy()

    # Create empty lists to store match results
    matches = []

    # For each expense, find potential matching transactions
    for _, expense in expenses.iterrows():
        # Filter transactions by date range
        date_min = expense["expense_date"] - date_threshold
        date_max = expense["expense_date"] + date_threshold
        date_filtered = transactions[
            (transactions["transaction_date"] >= date_min)
            & (transactions["transaction_date"] <= date_max)
        ].copy()

        if len(date_filtered) == 0:
            continue

        # Filter by amount similarity
        amount = expense["expense_cost"]
        date_filtered["amount_diff"] = (
            abs(date_filtered["transaction_cost"] - amount) / amount
        )
        potential_matches = date_filtered[
            date_filtered["amount_diff"] <= amount_threshold
        ]
        if len(potential_matches) > 0:

            # Get the best match (smallest amount difference)
            k = 1
            top_matches = potential_matches.sort_values("amount_diff").iloc[:k]
            for _, best_match in top_matches.iterrows():
                matches.append(
                    {
                        "expense_id": expense["expense_id"],
                        "transaction_hash": best_match["transaction_hash"],
                        "amount_diff": best_match["amount_diff"],
                        "cost_diff": abs(
                            expense["expense_cost"] - best_match["transaction_cost"]
                        ),
                        "expense_time_diff": abs(
                            expense["expense_date"] - best_match["transaction_date"]
                        ),
                    }
                )
    matches = pd.DataFrame(matches)
    assert matches.expense_id.is_unique, "Expense ids are not unique in matches"
    # assert matches.transaction_hash.is_unique, "Transaction hashes are not unique in matches"
    return matches


for threshold in [0.1, 0.01, 0.001]:
    print(f"Threshold: {threshold}")
    matched_transactions = match_transactions(
        expenses, transactions, amount_threshold=threshold
    )
    print(
        f"number of rows: {len(matched_transactions)}, max difference $ {matched_transactions['cost_diff'].max()} CLP"
    )

In [ ]:
matched_transactions = match_transactions(
    expenses, transactions, amount_threshold=0.001
)
matched_transactions = matched_transactions.merge(
    expenses, left_on="expense_id", right_on="expense_id", how="left"
).merge(
    transactions,
    left_on="transaction_hash",
    right_on="transaction_hash",
    how="right",
)
matched_transactions

In [ ]:
matched_transactions[
    matched_transactions.transaction_hash
    == "b03b6b7e3e2b8a70535488a40b6d30ec4aebd828a7a2161e4fb2de843ca2fc4e"
]

In [ ]:
def match_locations(
    transactions_df: pd.DataFrame,
    locations_df: gpd.GeoDataFrame,
    date_threshold: pd.Timedelta = pd.Timedelta("1m"),
) -> pd.DataFrame:
    # This is very slow, but it works
    # Need to rework if you want to use all your emails and expenses

    assert transactions_df[
        "transaction_hash"
    ].is_unique, "Transactions hashes are not unique"

    transactions = transactions_df.copy()
    locations = locations_df.copy()

    # Create empty lists to store match results
    matches = []

    # For each transaction, find potential matching locations
    for _, transaction in transactions.iterrows():
        # Filter locations by date range
        date_min = transaction["transaction_date"] - pd.Timedelta(date_threshold)
        date_max = transaction["transaction_date"] + pd.Timedelta(date_threshold)
        date_filtered = locations[
            (locations["location_date"] >= date_min)
            & (locations["location_date"] <= date_max)
        ].copy()
        date_filtered["time_diff"] = abs(
            date_filtered["location_date"] - transaction["transaction_date"]
        )

        if len(date_filtered) == 0:
            continue

        potential_matches = date_filtered
        if len(potential_matches) > 0:
            k = 1
            top_matches = potential_matches.sort_values("time_diff").iloc[:k]
            for _, best_match in top_matches.iterrows():
                matches.append(
                    {
                        "transaction_hash": transaction["transaction_hash"],
                        "location_id": best_match["location_id"],
                        "location_time_diff": best_match["time_diff"],
                    }
                )
    matches = pd.DataFrame(matches)
    assert (
        matches.transaction_hash.is_unique
    ), "Transaction hashes are not unique in matches"
    # assert matches.location_id.is_unique, "Location ids are not unique in matches"
    return matches


matched_locs = match_locations(transactions, locations, pd.Timedelta("5m"))
matched_transactions_locs = matched_locs.merge(
    locations, left_on="location_id", right_on="location_id", how="left"
).merge(
    matched_transactions,
    left_on="transaction_hash",
    right_on="transaction_hash",
    how="right",
)
matched_transactions_locs

In [ ]:
assert (
    transactions.shape[0] == matched_transactions_locs.shape[0]
), "Number of transactions do not match"

# Label classes is_shared

In [ ]:
matched_transactions_locs["expense_deleted_by_name"].value_counts(dropna=False)

In [ ]:
matched_transactions_locs["is_shared_feedback"] = (
    matched_transactions_locs["expense_deleted_by_name"] == "Diego"
)  # manual feedback in the app - when the user deletes the expense
matched_transactions_locs["is_shared"] = (
    matched_transactions_locs["expense_id"].notnull()
    | matched_transactions_locs["is_shared_feedback"]
)
matched_transactions_locs[matched_transactions_locs["is_shared_feedback"]]

In [ ]:
matched_transactions_locs.to_pickle(
    "processed/matched_transactions_locs.pkl"
)  # Save the matched transactions with locations